In [1]:
from pathlib import Path
import logging
import importlib

import rospy
from scipy.spatial.transform import Rotation as R
import numpy as np

from alr_sim.core import Scene

from grasp_benchmark.utils.ycb_utils import YCBLoader
from alr_simulation_tools.scene_utils import execute_grasping_sequence, create_sample_data, reset_scene

from grasping_benchmarks_ros.srv import (
    GraspPlanner,
    GraspPlannerResponse,
)

from grasp_benchmark.utils.ros_utils import (
    create_grasp_planner_request,
    calc_point_cloud_from_images
) 

pybullet build time: Oct 28 2022 16:11:27


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# define scene parameters

factory_string = "mj_beta"
object_pos = (0.5, 0.0, 0.2)
adhust_object_position = True
cam_pos = (0.5, 0.0, 1)
cam_quat = [0.7071067811865476, 0.0, 0.0, -0.7071067811865475]
# cam_quat = R.from_euler("xyz", (0, 0, -90), degrees=True).as_quat()[
#         [3, 0, 1, 2]
#     ].tolist(),
# cam_quat = R.from_euler("zyx", (-90, -30, 0), degrees=True).as_quat()[
#         [3, 0, 1, 2]
#     ].tolist(),
# cam_quat = (
#     R.from_euler("zyx", (90, 50, 0), degrees=True).as_quat()[[3, 0, 1, 2]].tolist()
# )
cam_height = 480
cam_width = 640
robot_pos = (0.0, 0.5, 0.2)
robot_quat = (0, 1, 0, 0)
render_mode = Scene.RenderMode.HUMAN    
wait_time = 0.5
move_duration = 2

#
# grasp_planner_service_id = "grconvnet_bench/grconvnet_grasp_planner_service"
# grasp_planner_service_id = "ggcnn_bench/ggcnn_grasp_planner_service"
grasp_planner_service_id = "contact_bench/contact_grasp_planner_service"


In [3]:
ycb_loader = YCBLoader(
    ycb_base_folder=Path.home() / "Documents" / "SF-ObjectDataset" / "YCB", factory_string="mj_beta"
)
# obj_ids = [p.name for p in ycb_loader.ycb_base_folder.iterdir() if p.is_dir()]
obj_id = "011_banana"

In [4]:
rospy.init_node("grasp_planner_client")

importlib.reload(logging)
logging.basicConfig(level=logging.INFO)

In [5]:

sim_obj = ycb_loader.get_ycb_object(pos = (0.5, 0.0, 0.2), quat = (0,1,0,0), obj_id=obj_id, name=obj_id)

results, scene, agent = create_sample_data(
    factory_string=factory_string,
    cam_pos=cam_pos,
    cam_quat=cam_quat,
    cam_height=cam_height,
    cam_width=cam_width,
    robot_pos=robot_pos,
    robot_quat=robot_quat,
    object_list=[sim_obj],
    target_obj_name=obj_id,
    render_mode=render_mode,
    wait_time=wait_time,
    move_duration=move_duration,
)

pc_points, pc_colors = calc_point_cloud_from_images(
    rgb_img=results["rgb_img"], depth_img=results["depth_img"], cam_intrinsics=results["cam_intrinsics"]
)

In [6]:
grasp_req = create_grasp_planner_request(
    rgb_img=results["rgb_img"],
    depth_img=results["depth_img"],
    seg_img=results["seg_img"],
    pc_points=pc_points,
    pc_colors=pc_colors,
    cam_pos=results["cam_pos"],
    cam_quat=results["cam_quat"],
    cam_intrinsics=results["cam_intrinsics"],
    cam_height=cam_height,
    cam_width=cam_width,
    num_of_candidates=10_000,
)

In [7]:
rospy.wait_for_service(grasp_planner_service_id, timeout=30.0)
grasp_planner = rospy.ServiceProxy(grasp_planner_service_id, GraspPlanner)

reply: GraspPlannerResponse = grasp_planner(grasp_req)

In [8]:
best_grasp = sorted(reply.grasp_candidates, key=lambda x: x.score.data)[-1]

grasp_pos = best_grasp.pose.pose.position
grasp_pos = (grasp_pos.x, grasp_pos.y, grasp_pos.z)

grasp_quat = best_grasp.pose.pose.orientation
grasp_quat = np.array((grasp_quat.w, grasp_quat.x, grasp_quat.y, grasp_quat.z))

grasp_rot = R.from_quat(grasp_quat[[1, 2, 3, 0]]).as_matrix()

In [9]:
hover_pos = grasp_pos + np.array([0, 0, 0.1])
agent.gotoCartPositionAndQuat(grasp_pos, grasp_quat)

[[ 0.094238    0.98190363 -0.16426948]
 [ 0.98183094 -0.11896363 -0.14783657]
 [-0.16470336 -0.14735304 -0.97527426]]


In [10]:
# execute_grasping_sequence(
#     agent = agent,
#     grasp_pos = grasp_pos,
#     grasp_quat = grasp_quat
# )

INFO:root:Going to home position (0.5, 0, 0.5)
INFO:root:Going to hover_ position [ 0.53091522 -0.06242675  0.04629843]
INFO:root:Opening gripper
INFO:root:Going to grasp position (0.5309152193367481, -0.06242675334215163, -0.0037015676498408645)
INFO:root:Closing gripper
INFO:root:Going to hover position [ 0.53091522 -0.06242675  0.04629843]
INFO:root:Going to home position (0.5, 0, 0.5)
INFO:root:Going to drop position (0, 0.5, 0.5)
INFO:root:Opening gripper
INFO:root:Closing gripper


In [ ]:
# reset_scene(factory_string, scene, agent)